In [7]:
from typing import TypedDict

from langchain.chat_models import init_chat_model
from langchain_community.chat_models.tongyi import ChatTongyi
from langgraph.graph import START, StateGraph

model = ChatTongyi(model="qwen-max", tags=["joke"], streaming=True)

class State(TypedDict):
      topic: str
      joke: str
      poem: str


def write_joke(state: State):
      topic = state["topic"]
      joke_response = model.invoke(
            [{"role": "user", "content": f"Write a joke about {topic}"}]
      )
      return {"joke": joke_response.content}


def write_poem(state: State):
      topic = state["topic"]
      poem_response = model.invoke(
            [{"role": "user", "content": f"Write a short poem about {topic}"}]
      )
      return {"poem": poem_response.content}


graph = (
      StateGraph(State)
      .add_node(write_joke)
      .add_node(write_poem)
      # write both the joke and the poem concurrently
      .add_edge(START, "write_joke")
      .add_edge(START, "write_poem")
      .compile()
)

In [8]:
# The "messages" stream mode returns a tuple of (message_chunk, metadata)
# where message_chunk is the token streamed by the LLM and metadata is a dictionary
# with information about the graph node where the LLM was called and other information
for msg, metadata in graph.stream(
    {"topic": "cats"},
    stream_mode="messages",  
):
    # Filter the streamed tokens by the langgraph_node field in the metadata
    # to only include the tokens from the write_poem node
    if msg.content and metadata["langgraph_node"] == "write_poem":
        print(msg.content, end="|", flush=True)

In| shadows| soft|,| they silently creep,
E|yes gleaming bright|, in night's deep| sleep.
Paws pat| the ground, a quiet| song,
Tails curl|ing high, where| they belong.

Wh|iskers twitch at| the slightest sound,
Guard|ians of their myst|ic round.
F|eline grace, in every leap|,
Cats, with| secrets, they softly| keep.|